In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
%load_ext autoreload
%autoreload 2
from final_project.smlm_3d.config.datasets import dataset_configs

from final_project.smlm_3d.config.datasets import dataset_configs
from final_project.smlm_3d.data.datasets import TrainingDataSet, ExperimentalDataSet, GenericDataSet, MultiTrainingDataset
from final_project.smlm_3d.workflow_v2 import concat_dataset_features

BOUND = 16
GenericDataSet.bound = BOUND

z_range = 1000
dataset = 'simulated_ideal_psf'
cfg = dataset_configs[dataset]['training']
# cfg['csv'] = cfg['csv'].replace('_filtered', '')
dataset = TrainingDataSet(cfg, fit_plane_z=True, transform_data=False, z_range=1000, split_data=True, add_noise=False, filter_emitters_proximity=False)


Reading img...
Loading /home/miguel/Projects/uni/data/smlm_3d/simulated_ideal_psf/training/bead_stack.ome.tif
Loading /home/miguel/Projects/uni/data/smlm_3d/simulated_ideal_psf/training/bead_stack.csv
using image sharpness
136 emitters before filtering
117 emitters after borders


In [29]:
import tensorflow as tf
train_dataset = tf.data.Dataset.from_tensor_slices(((dataset.data['train'][0][0], dataset.data['train'][0][1]), dataset.data['train'][1]))
val_dataset = tf.data.Dataset.from_tensor_slices(((dataset.data['val'][0][0], dataset.data['val'][0][1]), dataset.data['val'][1]))

In [30]:
from tensorflow.keras.preprocessing.image import smart_resize

def grayscale_to_rgb(x, z):
    img, coords = x
    img = tf.image.grayscale_to_rgb(img)
    return ((img), z)

def resize_img(x, z):
    img, coords = x
    img = smart_resize(img, (224, 224))
    return ((img, coords), z)    

train_dataset = train_dataset.map(resize_img)
train_dataset = train_dataset.map(grayscale_to_rgb)

train_dataset = train_dataset.shuffle(1000)
train_dataset = train_dataset.batch(512).prefetch(tf.data.experimental.AUTOTUNE)

val_dataset = val_dataset.map(resize_img)
val_dataset = val_dataset.map(grayscale_to_rgb)
val_dataset = val_dataset.batch(512).prefetch(tf.data.experimental.AUTOTUNE)


In [31]:
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras import Model

input_layer = Input((224, 224, 3))

vgg_head = tf.keras.applications.MobileNetV3Small(
    include_top=False,
    weights="imagenet",
    input_tensor=input_layer,
    input_shape=(224, 224, 3),
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)
vgg_head.trainable=False

vgg_model = Model(inputs=vgg_head.input, outputs=vgg_head.layers[-2].output)

x = vgg_model(input_layer)
x = Dense(512)(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
z = Dense(1)(x)
model = Model(inputs=vgg_model.input, outputs=z)

In [32]:
from tensorflow import keras
model.compile(loss='mean_squared_error',optimizer=keras.optimizers.Adam(learning_rate=0.1, decay=1e-6),metrics=['mean_absolute_error'])

In [33]:
model.fit(train_dataset, epochs=100, verbose=True, validation_data=val_dataset)

Epoch 1/100
32/32 [==============================] - 8s 194ms/step - loss: 17793386.0000 - mean_absolute_error: 2607.9509 - val_loss: 3171799.2500 - val_mean_absolute_error: 1527.0247
Epoch 2/100
32/32 [==============================] - 5s 164ms/step - loss: 803327.5625 - mean_absolute_error: 700.1238 - val_loss: 380000.4375 - val_mean_absolute_error: 525.2036
Epoch 3/100
32/32 [==============================] - 5s 164ms/step - loss: 367695.9375 - mean_absolute_error: 517.0323 - val_loss: 330448.1562 - val_mean_absolute_error: 498.4709
Epoch 4/100
32/32 [==============================] - 5s 164ms/step - loss: 349145.1250 - mean_absolute_error: 507.7116 - val_loss: 330755.5938 - val_mean_absolute_error: 498.7594
Epoch 5/100
32/32 [==============================] - 5s 164ms/step - loss: 343219.5000 - mean_absolute_error: 504.8997 - val_loss: 330455.1875 - val_mean_absolute_error: 498.4833
Epoch 6/100
32/32 [==============================] - 5s 164ms/step - loss: 340778.4062 - mean_absolu

32/32 [==============================] - 5s 165ms/step - loss: 333808.5312 - mean_absolute_error: 500.1877 - val_loss: 330553.1562 - val_mean_absolute_error: 498.5933
Epoch 67/100
32/32 [==============================] - 5s 165ms/step - loss: 333809.3750 - mean_absolute_error: 500.1992 - val_loss: 330593.0312 - val_mean_absolute_error: 498.6277
Epoch 68/100
32/32 [==============================] - 5s 165ms/step - loss: 333801.6562 - mean_absolute_error: 500.1963 - val_loss: 330559.3750 - val_mean_absolute_error: 498.5987
Epoch 69/100
32/32 [==============================] - 5s 165ms/step - loss: 333798.7812 - mean_absolute_error: 500.1951 - val_loss: 330534.4688 - val_mean_absolute_error: 498.5760
Epoch 70/100
32/32 [==============================] - 5s 165ms/step - loss: 333805.2812 - mean_absolute_error: 500.2043 - val_loss: 330477.0938 - val_mean_absolute_error: 498.5160
Epoch 71/100
32/32 [==============================] - 5s 166ms/step - loss: 333789.7500 - mean_absolute_error: 50